In [3]:
import os
import re
from docx import Document

# Folder containing the Word file(s)
folder_path = r"C:\Users\Madhan\Write Final"

# Expanded Italian accented character replacement map
char_map = {
    "à": "a", "À": "A",
    "è": "e", "È": "E",
    "é": "e", "É": "E",
    "ì": "i", "Ì": "I",
    "ò": "o", "Ò": "O",
    "ù": "u", "Ù": "U",
    "—":", "
    
}

# Words that should remain untouched
exceptional_words = {"μ, µ"}  # Add more as needed

def clean_text(text, location="", table_mode=False):
    # Skip if text is exactly an exceptional word
    if text.strip() in exceptional_words:
        return text
    
    changes = []
    
    # Replace non-breaking space
    if "\u00A0" in text:
        changes.append("non-breaking space")
        text = text.replace("\u00A0", " ")
    
    # Replace Italian accented characters
    for k, v in char_map.items():
        if k in text:
            changes.append(k)
            text = text.replace(k, v)
    
    # Remove all other non-ASCII except bullet (•) and comma (,)
    text, removed_chars = remove_unwanted_specials(text)
    if removed_chars:
        changes.extend(removed_chars)
    
    # Print changes
    if changes:
        print(f"{location}: {', '.join(sorted(set(changes)))}")
    
    return text

def remove_unwanted_specials(text):
    removed_chars = []
    new_text = ""
    for ch in text:
        # Keep ASCII chars, bullet • (U+2022), and comma
        if ch.isascii() or ch == "•" or ch == ",":
            new_text += ch
        else:
            removed_chars.append(ch)
    return new_text, removed_chars

# Process all .docx files
for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(".docx"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_name}")
        
        doc = Document(file_path)
        
        # Clean paragraphs
        for i, para in enumerate(doc.paragraphs):
            para.text = clean_text(para.text, f"Paragraph {i+1}")
        
        # Clean tables
        for ti, table in enumerate(doc.tables):
            for ri, row in enumerate(table.rows):
                for ci, cell in enumerate(row.cells):
                    cell.text = clean_text(cell.text, f"Table {ti+1} Row {ri+1} Col {ci+1}", table_mode=True)
        
        doc.save(file_path)
        print(f"Overwritten: {file_path}\n{'-'*50}")

print("All files processed.")


Processing file: NC_Global Distributed AI Computing Market_Write1.docx
Paragraph 2: —
Paragraph 3: à, è
Overwritten: C:\Users\Madhan\Write Final\NC_Global Distributed AI Computing Market_Write1.docx
--------------------------------------------------
All files processed.


In [2]:
import os
import re
from docx import Document

# Folder containing Word documents
folder_path = r"C:\Users\Madhan\Write Final"

# Replacement dictionary (keys will be treated as regex patterns)
replacements = {
    r"\bvendor\b": "company",
    r"\bvendors\b": "companies",
    r"\bVendor\b": "Company",
    r"\bVendors\b": "Companies",
    r"\bvendor\.\b": "company.",
    r"\bvendors\.\b": "companies.",
    r"\bVendor\.\b": "Company.",
    r"\bVendors\.\b": "Companies.",
    r"—": ", ",
    r" & ": " and ",
    r"&": " and ",
    r"–": "-",
    r"mergers and acquisitions\s*\(merger and acquisition\)": "mergers and acquisitions",
    r"\(merger and acquisition\)\s*mergers and acquisitions": "mergers and acquisitions",
    r"mergers and acquisitions\(merger and acquisition\)": "mergers and acquisitions",
    r"\(merger and acquisition\)mergers and acquisitions": "mergers and acquisitions",
    r"’s": "",
    r"’": "",
    r"'s": "",
    r"'": "",
    r"‘": "",
    r"“":"",
    r"”":"",
    r"For instance, In":"For instance, in",
    r"  ":" ",
    r"₹":"INR ",
    r"€":"EUR",
    r"ô":"o",
    r"\b(R&D)\b": "research and development",
    r"\b(R and D)\b": "research and development",
    r"\b(R & D)\b": "research and development",
    r"\b(M and A)\b": "mergers and acquisitions",
    r"\b(M&A)\b": "mergers and acquisitions",
    r"\b(M & A)\b": "mergers and acquisitions",
    r"\bR&D\b": "Research and development",
    r"\bR and D\b": "Research and development",
    r"\bR & D\b": "Research and development",
    r"\bM and A\b": "Mergers and acquisitions",
    r"\bM&A\b": "Mergers and acquisitions",
    r"\bM & A\b": "Mergers and acquisitions",
    r"\research and development (research and development)": "research and development",
     r"research and development\s*\(research and development\)": "research and development",
    r"\(research and development\)\s*research and development": "research and development",
    r"research and development\(research and development\)": "research and development",
    r"\(research and development\)research and development": "research and development"
}

# Process all .docx files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".docx") and not filename.startswith("~$"):  # Skip temp files
        file_path = os.path.join(folder_path, filename)
        doc = Document(file_path)
        changes = []

        for para in doc.paragraphs:
            for run in para.runs:
                text = run.text
                for pattern, replacement in replacements.items():
                    matches = list(re.finditer(pattern, text))
                    if matches:
                        changes.extend([(match.group(), replacement) for match in matches])
                        text = re.sub(pattern, replacement, text)
                run.text = text

        if changes:
            doc.save(file_path)
            print(f"\n✅ Updated file: {filename}")
            # Print unique replacements made in this file
            unique_changes = list(dict.fromkeys(changes))  # Remove duplicates while preserving order
            for before, after in unique_changes:
                print(f' - Replaced: "{before}" → "{after}"')
        else:
            print(f"\nNo changes needed: {filename}")


✅ Updated file: NC_Global Distributed AI Computing Market_Write1.docx
 - Replaced: "  " → " "
 - Replaced: "vendor" → "company"
 - Replaced: "vendors" → "companies"
 - Replaced: "Vendor" → "Company"
 - Replaced: "Vendors" → "Companies"


In [9]:
import os
import html
import re
import shutil
import xlsxwriter
from docx import Document

# Keywords to be flagged in HTML content
ERROR_KEYWORDS = [
    "—", "â", "vendor", "Vendor", "vendors", "Vendors", "&", "  ","   ", "199", "2000", "2001", "“","”", "Covid","COVID", " war ","Russia", "Ukraine",
    "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", 
    "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022",
    "R&D", "R and D", "M and A", "M&A", "R & D", "M & A", "&nbsp;", "&nbsp;&nbsp;",
    "&lt;", "&gt;", "&amp;", "&quot;", "&#39;", "&ensp;", "&emsp;","—","–","—","à", "è", "é", "INR", "EUR ", "₹","®", "concentrated", "concentrate",
    "ì", "ò", "ù", "\\,", "'", "©", "®", "™", "€", "£", "¥","bold", "italic","Ö"
    "§", "¶", "•", "¢", "¬", "µ", "°", "±", "×", "÷", "‰",
    "À", "È", "É", "Ì", "Ò", "Ù", "'s","'", "undefined", "Undefined", " .", "}","{","+","illion","Ø", "global market", "regional market","region market", "USD", "dollar","dollars"
]

def convert_to_html(content):
    """
    Convert content (only Description) into HTML format, handling lists and paragraphs.
    """
    html_output = []
    in_list = False

    # Split content into paragraphs, ensuring we're handling it line by line
    paragraphs = [line.strip() for line in content.splitlines() if line.strip()]
    
    if not paragraphs:
        return "<p>No content available</p>"  # Handle empty content gracefully

    for para in paragraphs:
        # Check if the line starts with a bullet point or dash (for list items)
        if para.startswith(("-", "•", "*")):
            if not in_list:
                html_output.append("<ul>")
                in_list = True
            html_output.append(f"<li>{html.escape(para[1:].strip())}</li>")
        else:
            if in_list:
                html_output.append("</ul>")
                in_list = False
            # Convert normal paragraph text
            html_output.append(f"<p>{html.escape(para)}</p>")
    
    if in_list:
        html_output.append("</ul>")
    
    return "\n".join(html_output)

def find_and_highlight_errors(text, workbook):
    """
    Finds error keywords in text and highlights them in red.
    """
    # Store the rich parts (default and error-colored text)
    rich_parts = []
    matches = []

    # Search for keywords and highlight them in red
    for keyword in ERROR_KEYWORDS:
        pattern = re.compile(re.escape(keyword), re.IGNORECASE)
        for match in pattern.finditer(text):
            matches.append((match.start(), match.end(), match.group()))

    # Sort matches by start position
    matches.sort(key=lambda x: x[0])
    last_pos = 0

    # Loop through the matches and apply rich formatting
    for start, end, word in matches:
        if start > last_pos:
            rich_parts.append(text[last_pos:start])  # Append normal text
        rich_parts.append(workbook.add_format({'font_color': 'red'}))  # Add red color format for the error word
        rich_parts.append(text[start:end])  # Append the error word
        last_pos = end

    if last_pos < len(text):
        rich_parts.append(text[last_pos:])  # Append the remaining text
    
    return rich_parts

def extract_docx_content(file_path):
    doc = Document(file_path)
    data = []
    current_heading = ""
    current_paragraphs = []

    for para in doc.paragraphs:
        style = para.style.name
        text = para.text.strip()

        if style in ["Heading 1", "Heading 2"]:
            if text == "AI Content":
                break
            if current_heading:
                combined = "\n".join(current_paragraphs).strip()
                if combined:  # Only add if there's content
                    data.append((current_heading, combined))
                current_paragraphs = []
            current_heading = text
        else:
            current_paragraphs.append(text)

    if current_heading and current_paragraphs:
        combined = "\n".join(current_paragraphs).strip()
        if combined:  # Only add if there's content
            data.append((current_heading, combined))

    return data

def save_to_excel_with_partial_color(data, output_path):
    workbook = xlsxwriter.Workbook(output_path)
    worksheet = workbook.add_worksheet("Word Data")

    header_format = workbook.add_format({'bold': True})
    normal_format = workbook.add_format({'font_color': 'black'})
    red_format = workbook.add_format({'font_color': 'red'})  # Red formatting for error text

    headers = ["Title", "Description", "HTML tags", "Error"]
    for col_num, header in enumerate(headers):
        worksheet.write(0, col_num, header, header_format)

    for row_num, (header, content) in enumerate(data, start=1):
        # Check for empty content
        if not content:
            print(f"⚠ Skipping row {row_num}: No content found for '{header}'")
            continue  # Skip if there's no content

        # Write the Title (no conversion, plain text)
        worksheet.write(row_num, 0, header)

        # Write the Description (column B) as plain text
        worksheet.write(row_num, 1, content)

        # Convert the Description (column B) to HTML
        html_content = convert_to_html(content)

        # Write the HTML content (column C)
        worksheet.write(row_num, 2, html_content)

        # Find and highlight errors in the HTML content for the HTML tags column (column C)
        highlighted_content = find_and_highlight_errors(html_content, workbook)

        # Write the HTML content with highlighted errors in the HTML tags column (column C)
        worksheet.write_rich_string(row_num, 2, *highlighted_content)

        # Write the error report (if any errors found) in the Error column (column D)
        found_errors = find_unwanted_keywords(html_content)
        if found_errors:
            error_msg = "⚠ Contains error-related text: " + ", ".join(sorted(set(found_errors), key=str.lower))
            worksheet.write(row_num, 3, error_msg)
        else:
            worksheet.write(row_num, 3, "", normal_format)

    workbook.close()

def find_unwanted_keywords(text):
    """
    Return a list of unwanted keywords found in the text.
    """
    return [word for word in ERROR_KEYWORDS if word.lower() in text.lower()]

def move_existing_excels_to_html(folder_path):
    html_folder = os.path.join(folder_path, "HTML")
    os.makedirs(html_folder, exist_ok=True)

    for file in os.listdir(folder_path):
        if file.endswith(".xlsx"):
            src = os.path.join(folder_path, file)
            dst = os.path.join(html_folder, file)
            try:
                shutil.move(src, dst)
                print(f"📁 Moved existing Excel file to HTML folder: {file}")
            except Exception as e:
                print(f"⚠ Failed to move {file}: {e}")

def main():
    folder_path = r"C:\Users\Madhan\Write Final"  # Change as needed

    # Step 1: Move existing .xlsx files before processing
    move_existing_excels_to_html(folder_path)

    # Step 2: Process .docx files and generate new .xlsx
    processed, failed = 0, 0

    for filename in os.listdir(folder_path):
        # Skip temporary files (e.g., those starting with ~$)
        if filename.startswith("~$"):
            continue

        if filename.endswith(".docx"):
            docx_path = os.path.join(folder_path, filename)
            try:
                content_data = extract_docx_content(docx_path)
                if content_data:  # Ensure there's content to process
                    excel_filename = os.path.splitext(filename)[0] + "_colored.xlsx"
                    excel_output_path = os.path.join(folder_path, excel_filename)
                    save_to_excel_with_partial_color(content_data, excel_output_path)
                    print(f"✅ Processed: {excel_filename}")
                    processed += 1
                else:
                    print(f"⚠ Skipping {filename}: No content found.")
                    failed += 1
            except Exception as e:
                print(f"❌ Failed to process {filename}: {e}")
                failed += 1

    print("\n📊 Summary:")
    print(f"✅ Successfully processed: {processed}")
    print(f"❌ Failed to process: {failed}")

if __name__ == "__main__":
    main()

📁 Moved existing Excel file to HTML folder: Global Real-Time AI Agent Assist Market 2025-2029_Write_colored.xlsx
✅ Processed: NC_Global Distributed AI Computing Market_Write1_colored.xlsx

📊 Summary:
✅ Successfully processed: 1
❌ Failed to process: 0


In [4]:
###finnal

In [5]:
import pandas as pd
import os
import shutil

# Define folder paths
write_files_folder = r"C:\Users\Madhan\Write Output\Write Files"
write_final_folder = r"C:\Users\Madhan\Write Final"
output_save_folder = r"C:\Users\Madhan\Write Final\Write Upload"

# Get Excel files
write_files = [f for f in os.listdir(write_files_folder) if f.endswith(('.xlsx', '.xls'))]
if not write_files:
    raise FileNotFoundError("❌ No Excel files found in Write Files folder.")

write_final_files = [f for f in os.listdir(write_final_folder) if f.endswith(('.xlsx', '.xls'))]
if not write_final_files:
    raise FileNotFoundError("❌ No Excel files found in Write Final folder.")

# Function to extract prefix up to the word "market"
def get_market_prefix(name):
    name = name.lower()
    index = name.find("market")
    if index != -1:
        return name[:index].strip().replace(" ", "_")
    return None

# Build mapping of prefixes
write_file_map = {get_market_prefix(f): f for f in write_files if get_market_prefix(f)}
write_final_map = {get_market_prefix(f): f for f in write_final_files if get_market_prefix(f)}

# Match based on shared prefix
matched_prefix = None
for prefix in write_file_map:
    if prefix in write_final_map:
        matched_prefix = prefix
        break

if not matched_prefix:
    raise FileNotFoundError("❌ No matching files based on 'Market' prefix found.")

matched_write_file = write_file_map[matched_prefix]
matched_final_file = write_final_map[matched_prefix]

print(f"\n✅ Matching files found for prefix '{matched_prefix.replace('_', ' ').title()}':")
print(f"→ Write Output file: {matched_write_file}")
print(f"→ Write Final file:  {matched_final_file}")

# File paths
write_output1_path = os.path.join(write_files_folder, matched_write_file)
write_final_path = os.path.join(write_final_folder, matched_final_file)

# Read Excel files (skip first row)
df_output1 = pd.read_excel(write_output1_path, header=None, skiprows=1)
df_final = pd.read_excel(write_final_path, header=None, skiprows=1)

print(f"\nInitial data shapes:\nWrite Output1: {df_output1.shape}\nWrite Final: {df_final.shape}")

# Drop unnecessary columns
df_final.drop(columns=[col for col in [1, 3] if col in df_final.columns], inplace=True)
if 2 in df_output1.columns:
    df_output1.drop(columns=[2], inplace=True)

print(f"\nAfter dropping columns:\nWrite Output1 shape: {df_output1.shape}\nWrite Final shape: {df_final.shape}")

# Filter rows
keep_starts = [
    "Market Drivers 1", "Market Challenges 1", "Market Trends 1",
    "Market Drivers 2", "Market Challenges 2", "Market Trends 2",
    "Market Drivers 3", "Market Challenges 3", "Market Trends 3",
    "Market Definition"
]
remove_starts = ["Five Force Analysis", "Market Segmentation by"]

def filter_rows(df, keep_starts, remove_starts):
    mask_keep = df[0].astype(str).str.startswith(tuple(keep_starts))
    mask_remove = ~df[0].astype(str).str.startswith(tuple(remove_starts))
    return df[mask_keep | mask_remove]

df_output1 = filter_rows(df_output1, keep_starts, remove_starts)
df_final = filter_rows(df_final, keep_starts, remove_starts)

print(f"\nAfter removing unwanted rows:\nWrite Output1 shape: {df_output1.shape}\nWrite Final shape: {df_final.shape}")

# Replace "Company Landscape" with "Vendor Landscape"
company_count = df_final[0].astype(str).str.contains("Company Landscape").sum()
vendor_before = df_final[0].astype(str).str.contains("Vendor Landscape").sum()

df_final[0] = df_final[0].replace("Company Landscape", "Vendor Landscape")

vendor_after = df_final[0].astype(str).str.contains("Vendor Landscape").sum()
print(f"\nReplaced 'Company Landscape' with 'Vendor Landscape' in Write Final Unique IDs")
print(f"Original 'Company Landscape' count: {company_count}")
print(f"'Vendor Landscape' count before replacement: {vendor_before}")
print(f"'Vendor Landscape' count after replacement: {vendor_after}")

# Ensure column 0 is string
df_output1[0] = df_output1[0].astype(str)
df_final[0] = df_final[0].astype(str)

# Matched IDs
matched_ids = set(df_output1[0]).intersection(set(df_final[0]))
matched_ids_list = sorted(matched_ids)

# Lookup dictionary
if 2 not in df_final.columns:
    raise KeyError("Expected column 2 not found after dropping.")
lookup_dict = df_final.drop_duplicates(subset=0).set_index(0)[2].to_dict()

# Update Write Output1
df_output1[1] = df_output1.apply(lambda row: lookup_dict.get(row[0], row[1]), axis=1)

# Add non-matching rows from final
non_matching_rows = df_final[~df_final[0].isin(df_output1[0])].copy()
non_matching_rows[1] = non_matching_rows[2]
non_matched_ids_list = sorted(non_matching_rows[0].unique())

# Unique ID analysis
output1_ids = df_output1[0].nunique()
final_ids = df_final[0].nunique()

# Final merge
final_merged = pd.concat([df_output1, non_matching_rows], ignore_index=True)

# Final summary
print(f"\nUnique IDs count (after filtering):")
print(f"Write Output1 unique IDs: {output1_ids}")
print(f"Write Final unique IDs: {final_ids}")
print(f"Matched Unique IDs: {len(matched_ids_list)}")
for uid in matched_ids_list:
    print(f"→ {uid}")

print(f"\nNon-matched Unique IDs in Write Final: {len(non_matched_ids_list)}")
for uid in non_matched_ids_list:
    print(f"→ {uid}")

print(f"\nFinal merged dataframe shape: {final_merged.shape}")

# --- TRANSPOSE COLUMNS A AND B INTO ROW 1 AND ROW 2 ---
row1 = final_merged[0].tolist()
row2 = final_merged[1].tolist()

# Replace all kinds of dashes "–" with "-" in Row 1
row1 = [str(item).replace("–", "-") for item in row1]

# Handle duplicate "Market Definition" in Row 1
def fix_market_definition_duplicates(items):
    seen = 0
    new_items = []
    for item in items:
        if item == "Market Definition":
            seen += 1
            if seen == 2:
                # Change second occurrence to " Market Definition"
                new_items.append(" Market Definition")
                continue
        new_items.append(item)
    return new_items

row1 = fix_market_definition_duplicates(row1)

# Create transposed DataFrame
transposed_df = pd.DataFrame([row1, row2])

# Save output with sheet name "data"
output_filename = f"{matched_prefix.title()}Market_Upload.xlsx"
output_path = os.path.join(output_save_folder, output_filename)
transposed_df.to_excel(output_path, index=False, header=False, sheet_name='data')

print(f"\n✅ Final merged file saved at:\n{output_path}")

# ✅ NEW STEP: Copy final file (without matching) to new location with no extension
no_ext_folder = r"C:\Users\Madhan\Write Final\Write Final Upload"
os.makedirs(no_ext_folder, exist_ok=True)

# Create no-extension filename from final saved Excel
no_ext_filename = os.path.splitext(output_filename)[0]  # remove .xlsx
no_ext_path = os.path.join(no_ext_folder, no_ext_filename)

shutil.copy2(output_path, no_ext_path)

print(f"\n✅ Final file copied without extension to:\n{no_ext_path}")


✅ Matching files found for prefix 'Global High-Density Polyethylene (Hdpe)':
→ Write Output file: Global High-Density Polyethylene (HDPE) Market 2025-2029_Write Output1_converted.xlsx
→ Write Final file:  Global High-Density Polyethylene (HDPE) Market 2025-2029_Write_colored.xlsx

Initial data shapes:
Write Output1: (193, 3)
Write Final: (17, 4)

After dropping columns:
Write Output1 shape: (193, 2)
Write Final shape: (17, 2)

After removing unwanted rows:
Write Output1 shape: (193, 2)
Write Final shape: (17, 2)

Replaced 'Company Landscape' with 'Vendor Landscape' in Write Final Unique IDs
Original 'Company Landscape' count: 1
'Vendor Landscape' count before replacement: 0
'Vendor Landscape' count after replacement: 1

Unique IDs count (after filtering):
Write Output1 unique IDs: 192
Write Final unique IDs: 16
Matched Unique IDs: 15
→ Bargaining power of buyers
→ Bargaining power of suppliers
→ Market Challenges 1
→ Market Definition
→ Market Drivers 1
→ Market Trends 1
→ Notes and c